# Set-up Siemens PLCs and TIA Portal to export process and metadata


### ⛳ Objective

The goal for this Jupyter Notebook are to understand and achieve the following:
1. Access a TIA Portal project data using TIA Openness API
2. Create an import of PLC Code Datablocks (DBs)
3. Additionally, you will also import other project metadata

The exported data will form the basis of the configuration of your AWS IoT SiteWise assets and Shopfloor connectivity (SFC) framework later in this workshop

### 🗝️ Key Concepts and refreshers
1. **Siemens TIA Portal**: The [Totally Integrated Automation (TIA) Portal](https://new.siemens.com/global/en/products/automation/industry-software/automation-software/tia-portal.html) is an engineering framework by Siemens that integrates automation and digitalization for designing, configuring, and commissioning automation systems.     
2. **TIA Openness API**: The [TIA Openness API](https://support.industry.siemens.com/cs/document/109759162/tia-portal-openess-interface-description) is an interface provided by Siemens that allows for programmatic control and automation of TIA Portal projects, enabling custom tool integration and automated engineering workflows.  
3. **Siemens PLCSim Advanced**: [Siemens PLCSim Advanced](https://new.siemens.com/global/en/products/automation/systems/industrial/plcsim-advanced.html) is a software tool that simulates Siemens PLCs, enabling virtual commissioning, testing, and troubleshooting of automation programs without physical hardware.  
   


### &#x2699; Configuration 
This section is to import all the helper python pacckages needed to use the notebook

### &#x1F4DA; Libraries and Helper Functions

In [ ]:
%%capture config_logs

!py -m pip install pythonnet ec2_metadata pickleshare

import sys, json
import plcsim_helper as plcsim
from scripts.s7tia.tia_openness_export import *
import Siemens.Engineering as tia
import Siemens.Engineering.Compiler as comp
import Siemens.Engineering.Download as dl

from pathlib import Path
from System.IO import DirectoryInfo, FileInfo


plcsim.load_plcsim_library()

## 1. Check Simulated PLCs are available and online  

For each of the five network interfaces in this Windows instance, a simulated PLC is created and the Network Interface Card (NIC) is attached to it. We will use this network card to connect to the simulted instance and interact with it.

<div style="padding-left: 2em;">&#x2611; Make sure you have 5 Simulated PLCs and they are active. This can also be checked in the PLCSim GUI</div>
 

In [ ]:
plcsim.check_all_plcs()

<div class="alert alert-block alert-info">
<span style="font-size: 1.5em;">⚠️</span> If you do not see 5 PLCs detected and online, then execute cell #1 in Troubleshooting section
</div>

## 2. Extract data from TIA Portal PLC Project

In this step we will extract the metadata from the PLC Code and also the configuration. This extracted data will be the foundation on which we will create our factory's asset model abstraction in AWS

### 2.1 Pre-requisite for data extraction from TIA Portal:

Execute the next cell to make sure pre-requisites for TIA Portal exports are fulfilled by checking the following:
<div style="padding-left: 2em;">&#x2611; TIA Portal is opened</div>
<div style="padding-left: 2em;">&#x2611; PLC Project is loaded</div>
<div style="padding-left: 2em;">&#x2611; TIA Openness connection is established</div>

In [ ]:
try:
    processes = tia.TiaPortal.GetProcesses()                    # Making a list of all running TIA processes
    process = processes[0]                                      # Just taking the first process as an example
    mytia = process.Attach()                                    # Establishing connection with the TIA 
    myproject = mytia.Projects[0]                               # Establishing connection with the TIA Project
except Exception as e:
    print("Unfortunately, This check did not pass!")
    print (f"\nReason:\n{e}")

<div class="alert alert-block alert-info">
<span style="font-size: 1.5em;">⚠️</span> If your checks failed, then execute cell #2 in the Troubleshooting section and repeat this step
</div>

### 2.3 Extract metadata from the TIA Portal Project

Now, from the TIA portal project we will use all the metadata that we have extracted to create **Unified Namespace (UNS)** model in the following chapter

In [ ]:

with open("tia_portal_meta.txt", "w") as file:                          # Location of the extracted data
    for device in myproject.Devices :
        #print(device.Name)
        for deviceItem in device.DeviceItems:
            attribute_names = ["Comment"]                               # Extracting the comment field from all PLCs in the project

            for attribute_name in attribute_names:
                attribute_value = deviceItem.GetAttribute(attribute_name)
        
        print(f"{device.Name}: {attribute_value}")    
        print(f"{device.Name}: {attribute_value}", file=file)
    
    

The output is a raw summary of certain project related information that we have extracted from the PLC project <span style="font-family: monospace;">reInventAutoWorkshop_V19</span>


<div class="alert alert-block alert-info padding-left: 2em;">
<span style="font-size: 1.5em;">💡</span>  To locate this data in TIA Project, go to <span style="font-family: monospace;">PLC Device->LeftClick:Properties->General Info</span> and browse through what is available
</div>

### 2.3 Export all available PLC data points from TIA Project

By executing the next cell, you will create an export of the TIA Project Data Blocks (DBs). This is where he process data in the PLCs are configured and stored. We are using the **TIA Openness API** to extract this data.

<div style="padding-left: 2em;">
&#x2611; Check the exported data by going to <span style="font-family: monospace;">C:/Users/Administrator/Documents/TIA-Export</span> and there you will find 5 folders one for each PLC with .xml files and a <span style="font-family: monospace;">project_automationml.aml</span> file.
</div>

In [ ]:
base_export_dir = Path(r"C://Users//Administrator//Documents//TIA-Export")      # Location of the exported data

export_automationml(base_export_dir, myproject)
export_all_groups_and_blocks_from_all_plcs(base_export_dir, myproject)


<div>
<span style="font-size: 1.5em;">🚀</span><b> Great Job!</b> You have now exported all S7 PLC Metadata using the TIA Openness API. Now its time to create metadata schema for models and assets in AWS IoT SiteWise and conguration for connectivity using SFC.
<p align="right">
        ⏭️<em> Now let's continue with Notebook 02 <em>
    </p></div>

## 🛠️ Troubleshooting

1. To start or reset simulated PLCs to factory settings, please execute next step and the execute the first step. 

In [ ]:
!powershell.exe -Command "Set-Location -Path C:\Users\Administrator\Desktop ; ./start_plcs.bat"

2. To start/restart TIA Portal, execute the cell below

In [ ]:
!powershell.exe -Command "Set-Location -Path C:\Users\Administrator\Desktop ; ./start_tia.bat"